In [1]:
# コード非表示 
from IPython.display import HTML

HTML("""
<button id="code-show-switch-btn">スクリプトを非表示にする</button>

<script>
var code_show = true;

function switch_display_setting() {
    var switch_btn = $("#code-show-switch-btn");
    if (code_show) {
        $("div.input").hide();
        code_show = false;
        switch_btn.text("スクリプトを表示する");
    }else {
        $("div.input").show();
        code_show = true;
        switch_btn.text("スクリプトを非表示にする");
    }
}

$("#code-show-switch-btn").click(switch_display_setting);
</script>
""")

# ライブラリ 

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.offline as offline
li_colors = px.colors.DEFAULT_PLOTLY_COLORS
offline.init_notebook_mode(connected=True)

# データ 

https://www.kaggle.com/datasets/vijayuv/onlineretail  
- InvouceNo：発注番号
- StockCode：商品番号
- Description：商品説明
- Quantity：購入個数
- InvouceDate：購入日時
- UnitPrice：商品単価
- CustomerID：顧客番号
- Country：国

In [3]:
df_raw = pd.read_csv('../input/csv/OnlineRetail.csv', encoding='shift-jis')

In [4]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
df_raw.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [7]:
df_raw.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900,4070,4223,NaN,23260,NaN,NaN,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,10/31/2011 14:41,NaN,NaN,United Kingdom
freq,1114,2313,2369,NaN,1114,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,NaN,4.611114,15287.690570,NaN
std,NaN,NaN,NaN,218.081158,NaN,96.759853,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,13953.000000,NaN
50%,NaN,NaN,NaN,3.000000,NaN,2.080000,15152.000000,NaN
75%,NaN,NaN,NaN,10.000000,NaN,4.130000,16791.000000,NaN


# 前処理 

In [8]:
df = df_raw.copy()

## カラム名変更

In [9]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [10]:
df.columns = ['invoice_no', 'item_no', 'item_description', 'quantity',
              'invoice_date', 'unit_price', 'customer_id', 'country']

In [11]:
df.columns

Index(['invoice_no', 'item_no', 'item_description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

## 欠損値処理

In [12]:
df.shape

(541909, 8)

In [13]:
df.dropna(inplace=True)

In [14]:
df.shape

(406829, 8)

##  異常値処理

In [15]:
df.describe()

,quantity,unit_price,customer_id
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [16]:
# 購入量と単価にマイナスが存在するので、0以上のみ抽出
df.query('quantity >= 0 & unit_price >= 0', inplace=True)

In [17]:
df.describe()

,quantity,unit_price,customer_id
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


## データ型変換 

In [18]:
df.dtypes

invoice_no           object
item_no              object
item_description     object
quantity              int64
invoice_date         object
unit_price          float64
customer_id         float64
country              object
dtype: object

In [19]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [20]:
df.dtypes

invoice_no                  object
item_no                     object
item_description            object
quantity                     int64
invoice_date        datetime64[ns]
unit_price                 float64
customer_id                float64
country                     object
dtype: object

In [21]:
df.head()

,invoice_no,item_no,item_description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [22]:
df.tail()

,invoice_no,item_no,item_description,quantity,invoice_date,unit_price,customer_id,country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [23]:
df.to_csv('../output/online_retail.csv', index=False)

# デシル分析

In [24]:
# 購入量
df['total_price'] = df['quantity'] * df['unit_price']

In [25]:
# 顧客ごとの購入量を算出
df_by_customer_id = df.groupby('customer_id')['total_price'].sum().reset_index()

In [26]:
df_by_customer_id

,customer_id,total_price
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
...,...,...
4334,18280.0,180.60
4335,18281.0,80.82
4336,18282.0,178.05
4337,18283.0,2094.88


In [27]:
# 分位点を求める
dict_quantile = df_by_customer_id.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).to_dict()

In [28]:
dict_quantile

{'customer_id': {0.1: 12905.6,
  0.2: 13505.6,
  0.3: 14110.4,
  0.4: 14708.2,
  0.5: 15299.0,
  0.6: 15882.8,
  0.7: 16481.2,
  0.8: 17080.4,
  0.9: 17686.800000000003},
 'total_price': {0.1: 156.56599999999997,
  0.2: 250.106,
  0.3: 351.658,
  0.4: 489.72400000000005,
  0.5: 674.45,
  0.6: 941.9419999999996,
  0.7: 1353.74,
  0.8: 2057.914,
  0.9: 3646.1640000000007}}

In [29]:
# 分位点で分ける関数を定義
def cal_decil(x, col, dict_quantile):
    if x <= dict_quantile[col][0.1]:
        return 10
    elif x <= dict_quantile[col][0.2]:
        return 9
    elif x <= dict_quantile[col][0.3]: 
        return 8
    elif x <= dict_quantile[col][0.4]: 
        return 7
    elif x <= dict_quantile[col][0.5]: 
        return 6
    elif x <= dict_quantile[col][0.6]: 
        return 5
    elif x <= dict_quantile[col][0.7]: 
        return 4
    elif x <= dict_quantile[col][0.8]: 
        return 3
    elif x <= dict_quantile[col][0.9]: 
        return 2
    else:
        return 1

In [30]:
# デシルランクの算出
df_by_customer_id['decil_rank'] = df_by_customer_id['total_price'].apply(cal_decil, args=('total_price', dict_quantile))

In [31]:
df_by_customer_id

,customer_id,total_price,decil_rank
0,12346.0,77183.60,1
1,12347.0,4310.00,1
2,12348.0,1797.24,3
3,12349.0,1757.55,3
4,12350.0,334.40,8
...,...,...,...
4334,18280.0,180.60,9
4335,18281.0,80.82,10
4336,18282.0,178.05,9
4337,18283.0,2094.88,2


In [32]:
df_by_customer_id.sort_values('total_price', ascending=False, ignore_index=True, inplace=True)

In [33]:
df_by_customer_id

,customer_id,total_price,decil_rank
0,14646.0,280206.02,1
1,18102.0,259657.30,1
2,17450.0,194550.79,1
3,16446.0,168472.50,1
4,14911.0,143825.06,1
...,...,...,...
4334,17956.0,12.75,10
4335,16454.0,6.90,10
4336,14792.0,6.20,10
4337,16738.0,3.75,10


In [34]:
# 累積和、累積比を算出
df_by_customer_id['cumsum'] = df_by_customer_id['total_price'].cumsum()
df_by_customer_id['cumper'] = df_by_customer_id['cumsum'] / df_by_customer_id['cumsum'].max()

In [35]:
df_by_customer_id.head()

,customer_id,total_price,decil_rank,cumsum,cumper
0,14646.0,280206.02,1,280206.02,0.031444
1,18102.0,259657.30,1,539863.32,0.060581
2,17450.0,194550.79,1,734414.11,0.082413
3,16446.0,168472.50,1,902886.61,0.101318
4,14911.0,143825.06,1,1046711.67,0.117457


In [36]:
df_by_customer_id.tail()

,customer_id,total_price,decil_rank,cumsum,cumper
4334,17956.0,12.75,10,8911391.054,0.999998
4335,16454.0,6.90,10,8911397.954,0.999999
4336,14792.0,6.20,10,8911404.154,1.000000
4337,16738.0,3.75,10,8911407.904,1.000000
4338,13256.0,0.00,10,8911407.904,1.000000


In [37]:
# デシルランク別　購入金額合計、累積購入金額比率、ユニーク顧客数
df_decil = df_by_customer_id.groupby('decil_rank').agg({'total_price': ['sum'],
                                                        'cumper': ['max'],
                                                        'customer_id': ['nunique']})

In [38]:
df_decil

,total_price,cumper,customer_id
,sum,max,nunique
decil_rank,,,
1,5469382.460,0.613751,434
2,1180055.001,0.746171,434
3,727103.640,0.827764,433
4,493107.450,0.883098,435
5,343792.411,0.921677,433
6,252901.481,0.950057,434
7,179857.951,0.970240,434
8,132117.870,0.985065,434


In [39]:
# カラム名変更
df_decil.columns = ['total_price', 'total_price_cumper', 'nunique_customer_id']

In [40]:
df_decil

,total_price,total_price_cumper,nunique_customer_id
decil_rank,,,
1,5469382.460,0.613751,434
2,1180055.001,0.746171,434
3,727103.640,0.827764,433
4,493107.450,0.883098,435
5,343792.411,0.921677,433
6,252901.481,0.950057,434
7,179857.951,0.970240,434
8,132117.870,0.985065,434
9,87145.480,0.994844,434


In [41]:
# 累積購入金額比率の小数点2位以下を偶数で丸める
df_decil['total_price_cumper'] *= 100
df_decil['total_price_cumper'] = df_decil['total_price_cumper'].round(2)

In [42]:
df_decil

,total_price,total_price_cumper,nunique_customer_id
decil_rank,,,
1,5469382.460,61.38,434
2,1180055.001,74.62,434
3,727103.640,82.78,433
4,493107.450,88.31,435
5,343792.411,92.17,433
6,252901.481,95.01,434
7,179857.951,97.02,434
8,132117.870,98.51,434
9,87145.480,99.48,434


In [43]:
# 購入金額比率
df_decil['total_price_per'] = df_decil['total_price'] / df_by_customer_id['total_price'].sum()
df_decil['total_price_per'] *= 100
df_decil['total_price_per'] = df_decil['total_price_per'].round(2)

In [44]:
df_decil

,total_price,total_price_cumper,nunique_customer_id,total_price_per
decil_rank,,,,
1,5469382.460,61.38,434,61.38
2,1180055.001,74.62,434,13.24
3,727103.640,82.78,433,8.16
4,493107.450,88.31,435,5.53
5,343792.411,92.17,433,3.86
6,252901.481,95.01,434,2.84
7,179857.951,97.02,434,2.02
8,132117.870,98.51,434,1.48
9,87145.480,99.48,434,0.98


In [45]:
# 1人あたりの購入金額
df_decil['mean_price'] = df_decil['total_price'] / df_decil['nunique_customer_id']

In [46]:
df_decil

,total_price,total_price_cumper,nunique_customer_id,total_price_per,mean_price
decil_rank,,,,,
1,5469382.460,61.38,434,61.38,12602.263733
2,1180055.001,74.62,434,13.24,2719.020740
3,727103.640,82.78,433,8.16,1679.223187
4,493107.450,88.31,435,5.53,1133.580345
5,343792.411,92.17,433,3.86,793.977855
6,252901.481,95.01,434,2.84,582.722306
7,179857.951,97.02,434,2.02,414.419242
8,132117.870,98.51,434,1.48,304.419055
9,87145.480,99.48,434,0.98,200.796037


In [47]:
# カラム選択
df_decil = df_decil[['total_price', 'total_price_per', 'total_price_cumper', 'mean_price']]

In [48]:
df_decil

,total_price,total_price_per,total_price_cumper,mean_price
decil_rank,,,,
1,5469382.460,61.38,61.38,12602.263733
2,1180055.001,13.24,74.62,2719.020740
3,727103.640,8.16,82.78,1679.223187
4,493107.450,5.53,88.31,1133.580345
5,343792.411,3.86,92.17,793.977855
6,252901.481,2.84,95.01,582.722306
7,179857.951,2.02,97.02,414.419242
8,132117.870,1.48,98.51,304.419055
9,87145.480,0.98,99.48,200.796037


In [49]:
df_decil.reset_index(inplace=True)

In [50]:
df_decil

,decil_rank,total_price,total_price_per,total_price_cumper,mean_price
0,1,5469382.460,61.38,61.38,12602.263733
1,2,1180055.001,13.24,74.62,2719.020740
2,3,727103.640,8.16,82.78,1679.223187
3,4,493107.450,5.53,88.31,1133.580345
4,5,343792.411,3.86,92.17,793.977855
5,6,252901.481,2.84,95.01,582.722306
6,7,179857.951,2.02,97.02,414.419242
7,8,132117.870,1.48,98.51,304.419055
8,9,87145.480,0.98,99.48,200.796037
9,10,45944.160,0.52,100.00,105.862120


In [51]:
df_decil.columns = ['デシルランク', '購入金額合計(円)', '購入金額比率(%)', '累積購入金額比率(%)', '1人あたりの購入金額(円)']

In [52]:
df_decil.style.background_gradient()

,デシルランク,購入金額合計(円),購入金額比率(%),累積購入金額比率(%),1人あたりの購入金額(円)
0,1,5469382.460000,61.380000,61.380000,12602.263733
1,2,1180055.001000,13.240000,74.620000,2719.020740
2,3,727103.640000,8.160000,82.780000,1679.223187
3,4,493107.450000,5.530000,88.310000,1133.580345
4,5,343792.411000,3.860000,92.170000,793.977855
5,6,252901.481000,2.840000,95.010000,582.722306
6,7,179857.951000,2.020000,97.020000,414.419242
7,8,132117.870000,1.480000,98.510000,304.419055
8,9,87145.480000,0.980000,99.480000,200.796037
9,10,45944.160000,0.520000,100.000000,105.862120


# RFM分析 

## RFM算出 

In [53]:
df_rfm = df.groupby('customer_id')[['invoice_date', 'total_price']].agg({'invoice_date': ['last', 'count'],
                                                                         'total_price': 'sum'}).reset_index()

In [54]:
df_rfm

customer_id        invoice_date       total_price
                                last count         sum
0        12346.0 2011-01-18 10:01:00     1    77183.60
1        12347.0 2011-12-07 15:52:00   182     4310.00
2        12348.0 2011-09-25 13:13:00    31     1797.24
3        12349.0 2011-11-21 09:51:00    73     1757.55
4        12350.0 2011-02-02 16:01:00    17      334.40
...          ...                 ...   ...         ...
4334     18280.0 2011-03-07 09:52:00    10      180.60
4335     18281.0 2011-06-12 10:53:00     7       80.82
4336     18282.0 2011-12-02 11:43:00    12      178.05
4337     18283.0 2011-12-06 12:02:00   756     2094.88
4338     18287.0 2011-10-28 09:29:00    70     1837.28

[4339 rows x 4 columns]

In [55]:
# カラム名変更
df_rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

In [56]:
df_rfm

,customer_id,recency,frequency,monetary
0,12346.0,2011-01-18 10:01:00,1,77183.60
1,12347.0,2011-12-07 15:52:00,182,4310.00
2,12348.0,2011-09-25 13:13:00,31,1797.24
3,12349.0,2011-11-21 09:51:00,73,1757.55
4,12350.0,2011-02-02 16:01:00,17,334.40
...,...,...,...,...
4334,18280.0,2011-03-07 09:52:00,10,180.60
4335,18281.0,2011-06-12 10:53:00,7,80.82
4336,18282.0,2011-12-02 11:43:00,12,178.05
4337,18283.0,2011-12-06 12:02:00,756,2094.88


In [57]:
df['invoice_date'].describe(datetime_is_numeric=True)

count                           397924
mean     2011-07-10 23:43:36.912475648
min                2010-12-01 08:26:00
25%                2011-04-07 11:12:00
50%                2011-07-31 14:39:00
75%                2011-10-20 14:33:00
max                2011-12-09 12:50:00
Name: invoice_date, dtype: object

In [58]:
max_date = df['invoice_date'].max()

In [59]:
max_date

Timestamp('2011-12-09 12:50:00')

In [60]:
# Recencyを算出
sr_recency = (max_date - df_rfm['recency']).dt.days
df_rfm['recency'] = sr_recency

In [61]:
df_rfm

,customer_id,recency,frequency,monetary
0,12346.0,325,1,77183.60
1,12347.0,1,182,4310.00
2,12348.0,74,31,1797.24
3,12349.0,18,73,1757.55
4,12350.0,309,17,334.40
...,...,...,...,...
4334,18280.0,277,10,180.60
4335,18281.0,180,7,80.82
4336,18282.0,7,12,178.05
4337,18283.0,3,756,2094.88


## RFMランク算出 

In [62]:
# 3指標それぞれの四分位点を求める
quantiles = df_rfm.quantile(q=[0.2, 0.4, 0.6, 0.8])
quantiles_dict = quantiles.to_dict()

In [63]:
quantiles_dict

{'customer_id': {0.2: 13505.6, 0.4: 14708.2, 0.6: 15882.8, 0.8: 17080.4},
 'recency': {0.2: 12.600000000000023, 0.4: 32.0, 0.6: 71.0, 0.8: 178.0},
 'frequency': {0.2: 14.0, 0.4: 29.0, 0.6: 58.0, 0.8: 121.0},
 'monetary': {0.2: 250.106,
  0.4: 489.72400000000005,
  0.6: 941.9419999999996,
  0.8: 2057.914}}

In [64]:
# RFMを分位点で分ける関数を定義
def cal_recency_rank(x, col, df):
    if x <= df[col][0.2]:
        return 5
    elif x <= df[col][0.4]:
        return 4
    elif x <= df[col][0.6]: 
        return 3
    elif x <= df[col][0.8]: 
        return 2
    else:
        return 1

def cal_frequency_rank(x, col, df):
    if x <= df[col][0.2]:
        return 1
    elif x <= df[col][0.4]:
        return 2
    elif x <= df[col][0.6]: 
        return 3
    elif x <= df[col][0.8]: 
        return 4
    else:
        return 5
    
def cal_monetary_rank(x, col, df):
    if x <= df[col][0.2]:
        return 1
    elif x <= df[col][0.4]:
        return 2
    elif x <= df[col][0.6]: 
        return 3
    elif x <= df[col][0.8]: 
        return 4
    else:
        return 5

In [65]:
# 各種スコアの算出
df_rfm['recency_rank'] = df_rfm['recency'].apply(cal_recency_rank, args=('recency', quantiles_dict))
df_rfm['frequency_rank'] = df_rfm['frequency'].apply(cal_frequency_rank, args=('frequency', quantiles_dict))
df_rfm['monetary_rank'] = df_rfm['monetary'].apply(cal_monetary_rank, args=('monetary', quantiles_dict))

# ３指標を統合したスコアの算出
df_rfm['rfm_rank'] = df_rfm['recency_rank'] + df_rfm['frequency_rank'] + df_rfm['monetary_rank']

In [66]:
# 欠損の有無を確認
df_rfm.isnull().sum()

customer_id       0
recency           0
frequency         0
monetary          0
recency_rank      0
frequency_rank    0
monetary_rank     0
rfm_rank          0
dtype: int64

In [67]:
df_rfm

,customer_id,recency,frequency,monetary,recency_rank,frequency_rank,monetary_rank,rfm_rank
0,12346.0,325,1,77183.60,1,1,5,7
1,12347.0,1,182,4310.00,5,5,5,15
2,12348.0,74,31,1797.24,2,3,4,9
3,12349.0,18,73,1757.55,4,4,4,12
4,12350.0,309,17,334.40,1,2,2,5
...,...,...,...,...,...,...,...,...
4334,18280.0,277,10,180.60,1,1,1,3
4335,18281.0,180,7,80.82,1,1,1,3
4336,18282.0,7,12,178.05,5,1,1,7
4337,18283.0,3,756,2094.88,5,5,5,15


## RFMランク表 

In [68]:
df_recency = df_rfm['recency_rank'].value_counts().sort_index(ascending=False).reset_index()
df_frequency = df_rfm['frequency_rank'].value_counts().sort_index(ascending=False).reset_index()
df_monetary = df_rfm['monetary_rank'].value_counts().sort_index(ascending=False).reset_index()

In [69]:
df_rfm_rank = df_recency.merge(df_frequency)
df_rfm_rank = df_rfm_rank.merge(df_monetary)

In [70]:
df_rfm_rank

,index,recency_rank,frequency_rank,monetary_rank
0,5,868,867,868
1,4,905,859,868
2,3,858,855,867
3,2,843,830,868
4,1,865,928,868


## RFM総合ランク表 

In [71]:
df_rfm_rank2 = df_rfm.groupby(['recency_rank', 'frequency_rank', 'monetary_rank'])['rfm_rank'].value_counts().to_frame()

In [72]:
df_rfm_rank2.columns = ['cnt_rfm_rank']

In [73]:
df_rfm_rank2 = df_rfm_rank2.reset_index().sort_values(['rfm_rank', 'recency_rank', 'frequency_rank', 'monetary_rank'], ascending=False)

In [74]:
df_rfm_rank2

,recency_rank,frequency_rank,monetary_rank,rfm_rank,cnt_rfm_rank
118,5,5,5,15,306
117,5,5,4,14,69
114,5,4,5,14,68
94,4,5,5,14,170
116,5,5,3,13,23
...,...,...,...,...,...
2,1,1,3,5,21
24,2,1,1,4,123
5,1,2,1,4,59
1,1,1,2,4,74


## RFM総合ランク別顧客数 

In [75]:
df_rfm_rank3 = pd.DataFrame(df_rfm['rfm_rank'].value_counts().sort_index(ascending=False), columns=['rfm_rank'])

In [76]:
df_rfm_rank3.style.background_gradient()

,rfm_rank
15,306
14,307
13,314
12,340
11,328
10,354
9,355
8,368
7,388
6,378


In [77]:
df_rfm_rank3['rfm_rank'].sum()

4339

## 1次元 

### Recency分布 

In [78]:
df_rfm['recency_rank'].value_counts().sort_index(ascending=False)

5    868
4    905
3    858
2    843
1    865
Name: recency_rank, dtype: int64

In [79]:
fig = px.histogram(df_rfm, x='recency', marginal='box')
fig.show()

### Frequency分布 

In [80]:
df_rfm['frequency_rank'].value_counts().sort_index(ascending=False)

5    867
4    859
3    855
2    830
1    928
Name: frequency_rank, dtype: int64

In [81]:
fig = px.histogram(df_rfm, x='frequency', marginal='box')
fig.show()

### Monetary分布 

In [82]:
df_rfm['monetary_rank'].value_counts().sort_index(ascending=False)

5    868
4    868
3    867
2    868
1    868
Name: monetary_rank, dtype: int64

In [83]:
fig = px.histogram(df_rfm, x='monetary', marginal='box')
fig.show()

## 2次元 

### Recency x Frequency 

In [84]:
# ランクごとのユニーク顧客数をカウント
df_rf = pd.pivot_table(df_rfm, index='recency_rank', columns='frequency_rank', values='customer_id', aggfunc='count')

In [85]:
df_rf.style.background_gradient(axis=None)

frequency_rank,1,2,3,4,5
recency_rank,,,,,
1,383,258,139,66,19
2,222,200,210,149,62
3,150,176,203,190,139
4,116,125,169,247,248
5,57,71,134,207,399


In [86]:
fig = px.scatter(df_rfm, x='recency', y='frequency', marginal_x='box', marginal_y='box')
fig.show()

### Recency x Monetary 

In [87]:
# ランクごとのユニーク顧客数をカウント
df_rf = pd.pivot_table(df_rfm, index='recency_rank', columns='monetary_rank', values='customer_id', aggfunc='count')

In [88]:
df_rf.style.background_gradient(axis=None)

monetary_rank,1,2,3,4,5
recency_rank,,,,,
1,360,286,133,65,21
2,178,189,228,171,77
3,158,176,200,185,139
4,113,140,181,236,235
5,59,77,125,211,396


In [89]:
fig = px.scatter(df_rfm, x='recency', y='monetary', marginal_x='box', marginal_y='box')
fig.show()

### Frequency x Monetary 

In [90]:
# ランクごとのユニーク顧客数をカウント
df_rf = pd.pivot_table(df_rfm, index='frequency_rank', columns='monetary_rank', values='customer_id', aggfunc='count')

In [91]:
df_rf.style.background_gradient(axis=None)

monetary_rank,1,2,3,4,5
frequency_rank,,,,,
1,591.000000,198.000000,81.000000,39.000000,19.000000
2,159.000000,424.000000,171.000000,61.000000,15.000000
3,96.000000,137.000000,386.000000,195.000000,41.000000
4,22.000000,104.000000,144.000000,397.000000,192.000000
5,nan,5.000000,85.000000,176.000000,601.000000


In [92]:
fig = px.scatter(df_rfm, x='frequency', y='monetary', marginal_x='box', marginal_y='box')
fig.show()

## 3次元 

### 相関行列ヒートマップ 

In [93]:
df_rfm[['recency', 'frequency', 'monetary']].corr().style.background_gradient(axis=None)

,recency,frequency,monetary
recency,1.000000,-0.206029,-0.122046
frequency,-0.206029,1.000000,0.422289
monetary,-0.122046,0.422289,1.000000


In [94]:
df_rfm[['recency_rank', 'frequency_rank', 'monetary_rank']].corr().style.background_gradient(axis=None)

,recency_rank,frequency_rank,monetary_rank
recency_rank,1.000000,0.483155,0.457862
frequency_rank,0.483155,1.000000,0.756411
monetary_rank,0.457862,0.756411,1.000000


### 可視化 

In [95]:
df_rfm_rank3 = df_rfm_rank3.reset_index()

In [96]:
df_rfm_rank2

,recency_rank,frequency_rank,monetary_rank,rfm_rank,cnt_rfm_rank
118,5,5,5,15,306
117,5,5,4,14,69
114,5,4,5,14,68
94,4,5,5,14,170
116,5,5,3,13,23
...,...,...,...,...,...
2,1,1,3,5,21
24,2,1,1,4,123
5,1,2,1,4,59
1,1,1,2,4,74


In [97]:
df_plot = df_rfm_rank2[df_rfm_rank2['rfm_rank'].isin([15, 14, 13, 4, 3])]


fig = px.scatter_3d(df_plot,
                    x='recency_rank', y='frequency_rank', z='monetary_rank',
                    color='rfm_rank', opacity=0.5, size='cnt_rfm_rank'
                    )
fig.show()